In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os

# Quantum Phase Amplification

## 1. Carga Completa

Hasta este Notebook hemos generado todo lo necesario para cargar la probabilidad y la función a integrar. Necesitamos ahora aplicar el algoritmo de **Quantum Phase Amplification** sobre nuestro sistema. 
Los pasos son los siguientes:

1. Partir de un estado de n+1 qbits:

$$|\Psi_{0}\rangle_{n+1}=|0\rangle_{n}\otimes|0\rangle_{1}=|0\rangle_{n+1}$$

2. Cargamos la probabilidad discretizada sobre los n primeros q-bits usando la puerta $\hat{P_{n}}$ que actúa sobre n qbits:

$$|\Psi_{1}\rangle_{n+1}=\left(\hat{P_{n}}\otimes I\right)|\Psi_{0}\rangle_{n+1}=\left(\hat{P_{n}}\otimes I\right)|0\rangle_{n}\otimes|0\rangle_{1}=\hat{P_{n}}|0\rangle_{n}\otimes I|0\rangle_{1}=\hat{P_{n}}|0\rangle_{n}\otimes |0\rangle_{1}=\sum_{x=0}^{2^{n}-1}\sqrt{p(x)}|x\rangle_{n}\otimes |0\rangle_{1}$$

3. Cargamos la función que queremos integrar usando la puerta $\hat{R_{n+1}}$ que actúa sobre los n+1 qbits:

$$|\Psi_{2}\rangle_{n+1}=\hat{R_{n+1}}|\Psi_{1}\rangle_{n+1}=\hat{R_{n+1}}\left(\hat{P_{n}}\otimes I\right)|\Psi_{0}\rangle_{n+1}$$

$$|\Psi_{2}\rangle_{n+1}=\sum_{x=0}^{2^{n}-1}|x\rangle_{n}\otimes\left(\sqrt{p(x)f(x)}|1\rangle+\sqrt{p(x)(1-f(x))}|0\rangle\right)$$

4. Reorganizamos los términos del siguiente modo:

$$a=\sum_{x=0}^{2^{n}-1}p(x)f(x)$$

$$|\Psi_{1}\rangle = \frac{1}{\sqrt{a}}\sum_{x=0}^{2^{n}-1}|x\rangle_{n}\otimes\sqrt{p(x)f(x)}|1\rangle$$

$$|\Psi_{0}\rangle = \frac{1}{\sqrt{1-a}}\sum_{x=0}^{2^{n}-1}|x\rangle_{n}\otimes\sqrt{p(x)(1-f(x))}|0\rangle$$

$$|\Psi_{2}\rangle_{n+1}=\sqrt{a}|\Psi_{1}\rangle+\sqrt{1-a}|\Psi_{0}\rangle$$

Hasta aquí lo que tenemos es la carga de la probabilidad y la función a integrar en el sistema cuántico. En las siguientes celdas se implementa todos estos pasos:


In [ ]:
sys.path.append('/home/gferro/PhaseAmplification/')

In [ ]:
from qat.core.console import display
from qat.qpus import LinAlg

In [ ]:
from dataloading_module import  CreateLoadFunctionGate, CreatePG
from kk import get_histogram

In [ ]:
#Probability function
def p(x):
    return x*x

def f(x):
    return np.sin(x)

### Carga Completa: Circuito

In [ ]:
from qat.lang.AQASM import Program

In [ ]:
nqbits = 4
nbins = 2**nqbits
a = 0
b = 1
#Discretization for the function domain
centers, probs = get_histogram(p, a, b, nbins)
#Discretizated function to load 
DiscretizedFunction = f(centers)

#Quantum Program
qprog = Program()
qbits = qprog.qalloc(nqbits+1)
#Create Probability loading gate
P_gate = CreatePG(probs)
qprog.apply(P_gate, qbits[:-1])
#Create Function loading gate
R_gate = CreateLoadFunctionGate(DiscretizedFunction)    
qprog.apply(R_gate, qbits)



In [ ]:
#Create the circuit from the program
circuit = qprog.to_circ()
    
#Display the circuit
#display(circuit, max_depth = depth)
%qatdisplay circuit

In [ ]:

#Create a Job from the circuit
#The integral is loaded in the amplitud of the last qbit!!
job = circuit.to_job(qubits = [nqbits])
    
#Import and create the linear algebra simulator
linalgqpu = LinAlg()
    
#Submit the job to the simulator LinAlg and get the results
result = linalgqpu.submit(job)
QP = []
States = []
#Print the results
for sample in result:
    print("State %s probability %s" % (sample.state, sample.probability))
    QP.append(sample.probability)
    States.append(str(sample.state))
        
print('Quantum Measurement: {}'.format(QP[1]))    
print('Integral: {}'.format(sum(DiscretizedFunction*probs)))    
print('Todo OK?: {}'.format(np.isclose(QP[1], sum(DiscretizedFunction*probs))))

## 2. Amplificación de Amplitud

Después de la carga de datos tenemos el sistema en el estado:

$$|\Psi_{2}\rangle_{n+1}=\sqrt{a}|\Psi_{1}\rangle+\sqrt{1-a}|\Psi_{0}\rangle$$

Donde $a=\sum_{x=0}^{2^{n}-1}p(x)f(x)$


Como la integral es la amplitud del estado $|\Psi_{1}\rangle$ el paso siguiente es intentar maximizar la probabilidad de que al medir obtengamos dicho estado aplicando el algoritmo de Groover. 

El algoritmo de Groover adapatado a amplificación de fase consiste en aplicar un número óptimo de veces $k$ el operador $\hat{Q}$. Este operador se define como:

$$\hat{Q}=\hat{U}_{|\Psi_{2}\rangle} \hat{U}_{|\Psi_{0}\rangle}$$

Los operadores $\hat{U}_{|\Psi_{2}\rangle}$ y $\hat{U}_{|\Psi_{0}\rangle}$ se construyen del siguiente modo:

$$\hat{U}_{|\Psi_{0}\rangle } = \hat{I} - 2|\Psi_{0}\rangle \langle \Psi_{0}|$$
$$\hat{U}_{|\Psi_{2}\rangle } = \hat{I} - 2|\Psi_{2}\rangle \langle \Psi_{2}|$$


### 2.1 Operador $\hat{U}_{|\Psi_{0}\rangle}$

Este operador se construye del siguiente modo. 


$$\hat{U}_{|\Psi_{0}\rangle } = \hat{I} - 2|\Psi_{0}\rangle \langle \Psi_{0}|$$



Como 
$$|\Psi_{2}\rangle_{n+1}=\sqrt{a}|\Psi_{1}\rangle+\sqrt{1-a}|\Psi_{0}\rangle$$

Si hacemos:

$$\hat{U}_{|\Psi_{0}\rangle} |\Psi_{2}\rangle_{n+1} = \sqrt{a}|\Psi_{1}\rangle-\sqrt{1-a}|\Psi_{0}\rangle$$

Esto es una reflexión en torno al eje definido por el estado $|\Psi_{1}\rangle$. El problema es intentar generar un circuito que me permita generar este operador. 

Una operación equivalente que sí es fácilmente circuitable es: 

$$(\hat{I_{n}}\otimes X)(\hat{I_{n}}\otimes Z)(\hat{I_{n}}\otimes X)$$

Como $|\Psi_{2}\rangle_{n+1}$ tambien se puede poner como:

$$|\Psi_{2}\rangle_{n+1}=\sum_{x=0}^{2^{n}-1}|x\rangle_{n}\otimes\left(\sqrt{p(x)f(x)}|1\rangle+\sqrt{p(x)(1-f(x))}|0\rangle\right)$$

Podemos aplicar el operador sobre el estado:

$$(\hat{I_{n}}\otimes X)(\hat{I_{n}}\otimes Z)(\hat{I_{n}}\otimes X)|\Psi_{2}\rangle_{n+1}$$

$$(\hat{I_{n}}\otimes X)(\hat{I_{n}}\otimes Z)(\hat{I_{n}}\otimes X)\sum_{x=0}^{2^{n}-1}|x\rangle_{n}\otimes\left(\sqrt{p(x)f(x)}|1\rangle+\sqrt{p(x)(1-f(x))}|0\rangle\right)$$

$$(\hat{I_{n}}\otimes X)(\hat{I_{n}}\otimes Z)\sum_{x=0}^{2^{n}-1}\hat{I_{n}}|x\rangle_{n}\otimes\left(\sqrt{p(x)f(x)}X|1\rangle+\sqrt{p(x)(1-f(x))}X|0\rangle\right)$$

Como $X|0\rangle=|1\rangle$ y $X|1\rangle=|0\rangle$

$$(\hat{I_{n}}\otimes X)(\hat{I_{n}}\otimes Z)\sum_{x=0}^{2^{n}-1}|x\rangle_{n}\otimes\left(\sqrt{p(x)f(x)}|0\rangle+\sqrt{p(x)(1-f(x))}|1\rangle\right)$$


$$(\hat{I_{n}}\otimes X)\sum_{x=0}^{2^{n}-1}\hat{I_{n}}|x\rangle_{n}\otimes\left(\sqrt{p(x)f(x)}Z|0\rangle+\sqrt{p(x)(1-f(x))}Z|1\rangle\right)$$

Como $Z|0\rangle=|0\rangle$ y $Z|1\rangle=-|1\rangle$

$$(\hat{I_{n}}\otimes X)\sum_{x=0}^{2^{n}-1}|x\rangle_{n}\otimes\left(\sqrt{p(x)f(x)}|0\rangle-\sqrt{p(x)(1-f(x))}|1\rangle\right)$$

$$\sum_{x=0}^{2^{n}-1}\hat{I_{n}}|x\rangle_{n}\otimes\left(\sqrt{p(x)f(x)}X|0\rangle-\sqrt{p(x)(1-f(x))}X|1\rangle\right)$$

Nuevamente $X|0\rangle=|1\rangle$ y $X|1\rangle=|0\rangle$

$$\sum_{x=0}^{2^{n}-1}|x\rangle_{n}\otimes\left(\sqrt{p(x)f(x)}|1\rangle-\sqrt{p(x)(1-f(x))}|0\rangle\right)$$

Este estado que acabamos de obtener es exactamente:

$$\hat{U}_{|\Psi_{0}\rangle} |\Psi_{2}\rangle_{n+1} = \sqrt{a}|\Psi_{1}\rangle-\sqrt{1-a}|\Psi_{0}\rangle$$

Por lo que podemos implementar fácilmente la reflexión en torno a $|\Psi_{0}\rangle$ como:

$$\hat{U}_{|\Psi_{0}\rangle } = \hat{I} - 2|\Psi_{0}\rangle \langle \Psi_{0}|=(\hat{I_{n}}\otimes X)(\hat{I_{n}}\otimes Z)(\hat{I_{n}}\otimes X)$$

Básicamente aplicamos sobre el qbit n+1 una puerta Z emparedada entre dos puertas X

#### 2.1.1 Implementación Matricial

Al final el algoritmo de amplificación de amplitud (**QAA**) es un caso particular del algoritmo de Groover por lo que todo lo aprendido para este algoritmo es fácilmnente aplicable a la amplificación de amplitud. 

El operador $\hat{U}_{|\Psi_{0}\rangle}$ juega el mismo papel que el oráculo en el algoritmo de Grover. Cuando estudiamos dicho algoritmo este oráculo lo habíamos implementado a través de una operación matricial de una matriz identidad donde el elemento de la diagonal correspondiente al estado sobre el que se hacía la reflexión se cambiaba de signo. 

En el caso de la **QAA** nos podemos traer tal cual la puerta **Reflexion_Gate** basada en la función **Reflection** que se implementó durante el estido del Algoritmo de Grover. En este caso el estado sobre el que se hacía la reflexión era uno de los $2^{n}$ posibles estados de la base de medidas. En el caso de la **QAA** el estado sobre el que se hace la reflexión es:

$$\hat{U}_{|\Psi_{0}\rangle}= \frac{1}{\sqrt{1-a}}\sum_{x=0}^{2^{n}-1}|x\rangle_{n}\otimes\sqrt{p(x)(1-f(x))}|0\rangle$$

Es decir lo hacemos sobre un estado en el que el último qbit siempre es cero!!!

Podemos no obstante aprovechar una puerta **U0** que utilicé en el Notebook: **04_QPA_Zalo** de cuando empecé a estudiar el algoritmo. Esta puerta se genera a partir de una matriz identidad en la que se cambian de 1 a -1 todos aquellos estados en los que el último qbit es cero!!!

In [ ]:
from qat.lang.AQASM import AbstractGate
def Uf0(n):
    """
    Implementa una reflexion en torno al estado 0: I-2|w>|0><w|<0|
    """
    #Matriz Identidad
    Identity = np.identity(2**n)
    #Creo 2|w>|1><w|<1|
    #Matriz de ceros
    #Zeroes = np.zeros([2**n, 2**n])
    #Rellenos solo aquellos del tipo |w>|1>
    for i in range(0, 2**n, 2):
        Identity[i,i] = -1
    return Identity#-Zeroes
U0 = AbstractGate("U0", [int], matrix_generator=Uf0, arity = lambda x: x)

Para analizar detalladamente su comportamiento vamos a implementar dos versiones del circuito sin la reflexion y con la reflexión para ver lo que sucede

In [ ]:
def DoCircuit(nqbits = 4, Reflexion = True):
    from qat.lang.AQASM import Program
    nbins = 2**nqbits
    a = 0
    b = 1
    #Discretization for the function domain
    centers, probs = get_histogram(p, a, b, nbins)
    #Discretizated function to load 
    DiscretizedFunction = f(centers) 

    #Quantum Program
    qprog = Program()
    qbits = qprog.qalloc(nqbits+1)
    #Create Probability loading gate
    P_gate = CreatePG(probs)
    qprog.apply(P_gate, qbits[:-1])
    #Create Function loading gate
    R_gate = CreateLoadFunctionGate(DiscretizedFunction)    
    qprog.apply(R_gate, qbits)
    if Reflexion:
        qprog.apply(U0(nqbits+1), qbits)
        

    #Create the circuit from the program
    circuit = qprog.to_circ()
    job = circuit.to_job()
    #Import and create the linear algebra simulator
    linalgqpu = LinAlg()
    #Submit the job to the simulator LinAlg and get the results
    result = linalgqpu.submit(job)
    QP = []
    States = []
    QA = []
    #Print the results
    for sample in result:
        print("State %s probability %s amplitude %s" % (sample.state, sample.probability, sample.amplitude))
        QP.append(sample.probability)
        States.append(str(sample.state))
        QA.append(sample.amplitude)
    QP = pd.Series(QP, name='Probability')  
    States = pd.Series(States, name='States')  
    QA = pd.Series(QA, name='Amplitude') 
    pdf = pd.concat([States, QP, QA], axis=1)
    return pdf, circuit

#### No implementamos Reflexion

In [ ]:
pdfNotReflexion, circuitNR = DoCircuit(Reflexion=False)

In [ ]:
%qatdisplay circuitNR

#### Implementamos Reflexion

In [ ]:
pdfReflexion, circuitR = DoCircuit()

In [ ]:
%qatdisplay circuitR

Según el razonamiento téorico si aplicamos la reflexión el circuito resultante debe verificar:

1. Los estados con el último qbit igual a 0 ($|x\rangle\otimes|0\rangle$ ) deben tener las amplitudes con el signo cambiado con respecto al circuito **sin la reflexión**
2. Los estados con el último qbit igual a 1 ($|x\rangle\otimes|1\rangle$ ) deben tener las amplitudes con el iguales con respecto al circuito **sin la reflexión**

In [ ]:
#Signo Cambiado
States0_NR = pdfNotReflexion[pdfNotReflexion['States'].str.extract(r'(\d)>')[0] == '0']
States0_R = pdfReflexion[pdfReflexion['States'].str.extract(r'(\d)>')[0] == '0']
print(np.isclose(States0_NR['Amplitude'], -States0_R['Amplitude']))
print(np.isclose(States0_NR['Amplitude'], -States0_R['Amplitude']).all())

In [ ]:
#Signo Cambiado
States1_NR = pdfNotReflexion[pdfNotReflexion['States'].str.extract(r'(\d)>')[0] == '1']
States1_R = pdfReflexion[pdfReflexion['States'].str.extract(r'(\d)>')[0] == '1']

print(np.isclose(States1_NR['Amplitude'], States1_R['Amplitude']))
print(np.isclose(States1_NR['Amplitude'], States1_R['Amplitude']).all())

#### 2.1.2 Implementación Circuital

Para realizar la implementación circuital vamos a utilzar la siguiente expresión como base:

$$\hat{U}_{|\Psi_{0}\rangle } = \hat{I} - 2|\Psi_{0}\rangle \langle \Psi_{0}|=(\hat{I_{n}}\otimes X)(\hat{I_{n}}\otimes Z)(\hat{I_{n}}\otimes X)$$

In [ ]:
from qat.lang.AQASM import AbstractGate, QRoutine, X, Z
def Reflexion_generator(N):
    """
    Implementa una reflexion en torno al estado |\Phi_0>: I-2|\Phi_0><Phi_0|
    """
    qrout = QRoutine()
    qbits = qrout.new_wires(N)
    qrout.apply(X, qbits[-1])
    qrout.apply(Z, qbits[-1])
    qrout.apply(X, qbits[-1])
    
    return qrout#-Zeroes
Phi_0 = AbstractGate("Phi_0", [int])
Phi_0.set_circuit_generator(Reflexion_generator)

In [ ]:
def DoCircuit(nqbits = 4, Reflexion = True):
    from qat.lang.AQASM import Program
    nbins = 2**nqbits
    a = 0
    b = 1
    #Discretization for the function domain
    centers, probs = get_histogram(p, a, b, nbins)
    #Discretizated function to load 
    DiscretizedFunction = f(centers) 

    #Quantum Program
    qprog = Program()
    qbits = qprog.qalloc(nqbits+1)
    #Create Probability loading gate
    P_gate = CreatePG(probs)
    qprog.apply(P_gate, qbits[:-1])
    #Create Function loading gate
    R_gate = CreateLoadFunctionGate(DiscretizedFunction)    
    qprog.apply(R_gate, qbits)
    if Reflexion:
        qprog.apply(Phi_0(nqbits+1), qbits)
        

    #Create the circuit from the program
    circuit = qprog.to_circ()
    job = circuit.to_job()
    #Import and create the linear algebra simulator
    linalgqpu = LinAlg()
    #Submit the job to the simulator LinAlg and get the results
    result = linalgqpu.submit(job)
    QP = []
    States = []
    QA = []
    #Print the results
    for sample in result:
        #print("State %s probability %s amplitude %s" % (sample.state, sample.probability, sample.amplitude))
        QP.append(sample.probability)
        States.append(str(sample.state))
        QA.append(sample.amplitude)
    QP = pd.Series(QP, name='Probability')  
    States = pd.Series(States, name='States')  
    QA = pd.Series(QA, name='Amplitude') 
    pdf = pd.concat([States, QP, QA], axis=1)
    return pdf, circuit

In [ ]:
pdfNotReflexion, circuitNR = DoCircuit(Reflexion=False)
pdfReflexion, circuitR = DoCircuit()

In [ ]:
%qatdisplay circuitR

In [ ]:
%qatdisplay circuitNR

In [ ]:
pdfReflexion

In [ ]:
#Signo Cambiado
States0_NR = pdfNotReflexion[pdfNotReflexion['States'].str.extract(r'(\d)>')[0] == '0']
States0_R = pdfReflexion[pdfReflexion['States'].str.extract(r'(\d)>')[0] == '0']
print(np.isclose(States0_NR['Amplitude'], -States0_R['Amplitude']))
print(np.isclose(States0_NR['Amplitude'], -States0_R['Amplitude']).all())

In [ ]:
#Signo Cambiado
States1_NR = pdfNotReflexion[pdfNotReflexion['States'].str.extract(r'(\d)>')[0] == '1']
States1_R = pdfReflexion[pdfReflexion['States'].str.extract(r'(\d)>')[0] == '1']

print(np.isclose(States1_NR['Amplitude'], States1_R['Amplitude']))
print(np.isclose(States1_NR['Amplitude'], States1_R['Amplitude']).all())

### 2.2 Operador $\hat{U}_{|\Psi_{2}\rangle}$

El operador $\hat{U}_{|\Psi_{2}\rangle}$ se basa en el operador difusor de Groover.

En el **algoritmo de Groover** el estado de partida era una superposición equiprobable de los autoestados:

$$\Psi = H^{\otimes n} |0\rangle_{n}$$ y el difusor se definía como:

$$\hat{D} = \hat{I}-2|\Psi \rangle \langle \Psi| = H^{\otimes n}(\hat{I}-2|0\rangle
\langle0|)H^{\otimes n}$$

Donde $$\hat{D}_{0} = \hat{I}-2|0\rangle \langle0|$$

Es una reflexión en torno al estado perpendicular al estado $|0\rangle_{n}$

En el caso de la amplificación de amplitud el operador $\hat{U}_{|\Psi_{2}\rangle}$:

$$\hat{U}_{|\Psi_{2}\rangle } = \hat{I} - 2|\Psi_{2}\rangle \langle \Psi_{2}|$$

lo podemos obtener de un modo similar al difusor de Groover si tenemos en cuenta que

$$|\Psi_{2}\rangle_{n+1}=\hat{R_{n+1}}\hat{P_{n}}|0\rangle_{n+1}$$

Y como $\hat{R_{n+1}}$, $\hat{P_{n}}$ son dos operadores unitarios:


Lo podemos poner como: 
$$\hat{U}_{|\Psi_{2}\rangle } =\hat{R_{n+1}}\hat{P_{n}}\hat{D}_{0} \hat{P_{n}}^{\dagger} \hat{R_{n+1}}^{\dagger}$$

Necesitamos implementar: 

$$\hat{D}_{0} = \hat{I}-2|0\rangle \langle0|$$


#### 2.2.1 Implementación Matricial $\hat{D}_{0}$

La implementación matricial del Operador $\hat{D}_{0}$:

$$\hat{D}_{0} = \hat{I}-2|0\rangle \langle0|$$

la tenemos de cuando estudiamos el algoritmo de Groover.

In [ ]:
from qat.lang.AQASM import AbstractGate
def Reflection(n, state, Positive=True):
    """
    Implementa una matriz de reflexion de dimensión n en torno a un estado dado.
    Positive:
        * True: I-2|w><w|
        * False: 2|w><w|-I
    """
    #Matriz Identidad
    Identity = np.identity(2**n)
    Identity[state, state] =-1
    if Positive:
        return Identity
    else:
        return -Identity
#Creo una puerta utilizando el circuito    
Reflexion_Gate = AbstractGate(
    "Reflexion", 
    [int, int, bool], 
    matrix_generator=Reflection,
    arity = lambda x, y, z: x
)

In [ ]:
def DoCircuit(nqbits = 4, Reflexion = True):
    from qat.lang.AQASM import Program
    nbins = 2**nqbits
    a = 0
    b = 1
    #Discretization for the function domain
    centers, probs = get_histogram(p, a, b, nbins)
    #Discretizated function to load 
    DiscretizedFunction = f(centers) 

    #Quantum Program
    qprog = Program()
    qbits = qprog.qalloc(nqbits+1)
    #Create Probability loading gate
    P_gate = CreatePG(probs)
    qprog.apply(P_gate, qbits[:-1])
    #Create Function loading gate
    R_gate = CreateLoadFunctionGate(DiscretizedFunction)    
    qprog.apply(R_gate, qbits)
    if Reflexion:
        qprog.apply(Reflexion_Gate(nqbits+1, 0, True), qbits)
        

    #Create the circuit from the program
    circuit = qprog.to_circ()
    job = circuit.to_job()
    #Import and create the linear algebra simulator
    linalgqpu = LinAlg()
    #Submit the job to the simulator LinAlg and get the results
    result = linalgqpu.submit(job)
    QP = []
    States = []
    QA = []
    #Print the results
    for sample in result:
        print("State %s probability %s amplitude %s" % (sample.state, sample.probability, sample.amplitude))
        QP.append(sample.probability)
        States.append(str(sample.state))
        QA.append(sample.amplitude)
    QP = pd.Series(QP, name='Probability')  
    States = pd.Series(States, name='States')  
    QA = pd.Series(QA, name='Amplitude') 
    pdf = pd.concat([States, QP, QA], axis=1)
    return pdf, circuit

In [ ]:
pdfNotReflexion, circuitNR = DoCircuit(Reflexion=False)
pdfReflexion, circuitR = DoCircuit()

In [ ]:
%qatdisplay circuitR

In [ ]:
%qatdisplay circuitNR

En este caso solo hacemos una reflexion en torno al estado $|0\rangle$ por lo que la diferencia entre ambos estados estará en este autoestado cuya amplitud tiene que se opueesta en el caso del circuito que tiene la reflexión

In [ ]:
print('Las amplitudes del estado |0> son opuestas: {}'.format(
    pdfNotReflexion.loc[0]['Amplitude'] == - pdfReflexion.loc[0]['Amplitude']
))
print('Las amplitudes de los estados distintos de |0> son iguales: {}'.format(
    all(pdfNotReflexion[1:]['Amplitude'] == pdfReflexion[1:]['Amplitude'])
))

#### 2.2.2 Implementación Circuital $\hat{D}_{0}$

Se puede demostrar que la implementación Circuital del Operador $\hat{D}_{0}$ es:

$$\hat{D}_{0} = \hat{I}-2|0\rangle \langle0|= \hat{X}^{\otimes n} c^{n-1}Z \hat{X}^{\otimes n}$$

donde $c^{n-1}Z$ es una puerta Z controlada por los **n-1** qbits sobre el qbit **n-esimo**

Vamos a demostrar esto:

Partimos de un estado de **n-qbits** genéricos: $|\Psi_{0}\rangle = |q_{0}q_{1}...q_{n}\rangle$ donde 
$|q_i>=a_i|0\rangle+b_i|1\rangle$

Si desarrollamos los productos del estado inicial:


$$|q_{0}q_{1}...q_{n}\rangle = a_0a_1...a_{n-1}a_n|0\rangle_{n} \;+\;  a_0a_1...a_{n-1}b_n|0\rangle_{n-1}|1\rangle_{1}\; + \; ...$$

Donde los puntos suspensivos indican todos los desarrollos del producto que no nos interesan


Aplicamos $\hat{X}^{\otimes n}$:

$$\hat{X}^{\otimes n}|q_{0}q_{1}...q_{n}\rangle = \prod_{i=0}^{n} \hat{X} |q_i\rangle = \prod_{i=0}^{n} \hat{X}(a_i|0\rangle+b_i|1\rangle) =\prod_{i=0}^{n} (a_i\hat{X}|0\rangle+b_i\hat{X}|1\rangle)= \prod_{i=0}^{n} (a_i|1\rangle+b_i|0\rangle)$$

A continuación aplicamos la puerta Z controlada:
$$c^{n-1}Z \hat{X}^{\otimes n}|q_{0}q_{1}...q_{n}\rangle =  c^{n-1}\hat{Z}\prod_{i=0}^{n}(a_i|1\rangle+b_i|0\rangle)$$

La puerta controlada Z está controlada por lo n-1 primeros qbits y solo se aplicará cuando los **n-1** qbits sean $|1\rangle$. En todos los demás casos **NO** se aplicará: es decir:

$$c^{n-1}Z \hat{X}^{\otimes n}|q_{0}q_{1}...q_{n}\rangle = |a_0a_1...a_{n-1}|1\rangle_{n-1}(a_nZ|1\rangle+b_nZ|0\rangle) + \; ...$$
$$=a_0a_1...a_{n-1}|1\rangle_{n-1}(-a_n|1\rangle+b_n|0\rangle) + \; ...$$
$$ = - a_0a_1...a_{n-1}a_n|1\rangle_{n} \; + \; a_0a_1...a_{n-1}b_{n}|1\rangle_{n-1}|0\rangle \; + ... $$

Donde los puntos suspensivos representan estados que no se van a ver afectados por la aplicación controlada de la puerta Z. Además hemos usado que: $\hat{Z}|0\rangle = |0\rangle$ y $\hat{Z}|1\rangle = -|1\rangle$


Si ahora volvemos a aplicar la puerta $\hat{X}^{\otimes n}$:

$$\hat{X}^{\otimes n}c^{n-1}Z \hat{X}^{\otimes n}|q_{0}q_{1}...q_{n}\rangle = - a_0a_1...a_{n-1}a_n\hat{X}^{\otimes n}|1\rangle_{n} \; + \; a_0a_1...a_{n-1}b_{n}\hat{X}^{\otimes n-1}|1\rangle_{n-1}\hat{X}|0\rangle \; + ...$$
$$=- a_0a_1...a_{n-1}a_n|0\rangle_{n} \; + a_0a_1...a_{n-1}b_{n}|0\rangle_{n-1}|1\rangle+\; ...$$

Así pues la única diferencia entre: $|q_{0}q_{1}...q_{n}\rangle$ y $\hat{X}^{\otimes n}c^{n-1}Z \hat{X}^{\otimes n}|q_{0}q_{1}...q_{n}\rangle$ es que la componente $|0\rangle_{n}$ cambia de signo (todas las demas componentes son exactamente la misma). Es decir una reflexión en torno al estado $|0\rangle_{n}$

In [ ]:
from qat.lang.AQASM import AbstractGate, QRoutine, X, Z
def Reflexion_generator0(N):
    """
    Implementa una reflexion en torno al estado 0: I-2|0><0|
    """
    qrout = QRoutine()
    qbits = qrout.new_wires(N)
    for i in range(N):
        qrout.apply(X, qbits[i])
    #Controlled Z gate by n-1 first qbits
    cZ = 'Z'+ '.ctrl()'*(len(qbits)-1)
    qrout.apply(eval(cZ), qbits[:-1], qbits[-1])
    for i in range(N):
        qrout.apply(X, qbits[i])
    
    return qrout#-Zeroes
D_0 = AbstractGate("D_0", [int])
D_0.set_circuit_generator(Reflexion_generator0)

In [ ]:
def DoCircuit(nqbits = 4, Reflexion = True):
    from qat.lang.AQASM import Program
    nbins = 2**nqbits
    a = 0
    b = 1
    #Discretization for the function domain
    centers, probs = get_histogram(p, a, b, nbins)
    #Discretizated function to load 
    DiscretizedFunction = f(centers) 

    #Quantum Program
    qprog = Program()
    qbits = qprog.qalloc(nqbits+1)
    #Create Probability loading gate
    P_gate = CreatePG(probs)
    qprog.apply(P_gate, qbits[:-1])
    #Create Function loading gate
    R_gate = CreateLoadFunctionGate(DiscretizedFunction)    
    qprog.apply(R_gate, qbits)
    if Reflexion:
        qprog.apply(D_0(nqbits+1), qbits)

    #Create the circuit from the program
    circuit = qprog.to_circ()
    job = circuit.to_job()
    #Import and create the linear algebra simulator
    linalgqpu = LinAlg()
    #Submit the job to the simulator LinAlg and get the results
    result = linalgqpu.submit(job)
    QP = []
    States = []
    QA = []
    #Print the results
    for sample in result:
        #print("State %s probability %s amplitude %s" % (sample.state, sample.probability, sample.amplitude))
        QP.append(sample.probability)
        States.append(str(sample.state))
        QA.append(sample.amplitude)
    QP = pd.Series(QP, name='Probability')  
    States = pd.Series(States, name='States')  
    QA = pd.Series(QA, name='Amplitude') 
    pdf = pd.concat([States, QP, QA], axis=1)
    return pdf, circuit

In [ ]:
pdfNotReflexion, circuitNR = DoCircuit(Reflexion=False)
pdfReflexion, circuitR = DoCircuit()

In [ ]:
%qatdisplay circuitR --depth 0

In [ ]:
%qatdisplay circuitNR --depth 0

In [ ]:
print('Las amplitudes del estado |0> son opuestas: {}'.format(
    pdfNotReflexion.loc[0]['Amplitude'] == - pdfReflexion.loc[0]['Amplitude']
))
print('Las amplitudes de los estados distintos de |0> son iguales: {}'.format(
    all(pdfNotReflexion[1:]['Amplitude'] == pdfReflexion[1:]['Amplitude'])
))

In [ ]:
pdfNotReflexion

In [ ]:
pdfReflexion

El Notebook se me hace muy grande. Continuo en el siguiente

#### 2.2.3 Implementación Circuital $\hat{U}_{|\Psi_2}$

Nos queda implementar el Difusor:
$$\hat{U}_{|\Psi_{2}\rangle } = \hat{I} - 2|\Psi_{2}\rangle \langle \Psi_{2}|$$

Como ya tenemos implementado

$$\hat{D}_{0} = \hat{I}-2|0\rangle \langle0|$$

y sabemos que:

$$|\Psi_{2}\rangle_{n+1}=\hat{R_{n+1}}\hat{P_{n}}|0\rangle_{n+1}$$

La forma rápida de implementar el operador Difusor: $\hat{U}_{|\Psi_{2}\rangle }$

$$\hat{U}_{|\Psi_2\rangle} = \hat{R_{n+1}}\hat{P_{n}}\hat{D}_{0} \hat{P_{n}}^{\dagger} \hat{R_{n+1}}^{\dagger}$$

Este operador se puede interpretar como una reflexión en torno al estado perpendicular a $|\Psi_{2}\rangle$


In [ ]:
def LoadDifusorGate(N, ProbabilityArray, FunctionArray):
    from dataloading_module import  CreateLoadFunctionGate, CreatePG
    P_gate = CreatePG(ProbabilityArray)
    R_gate = CreateLoadFunctionGate(FunctionArray)
    def Difusor(nqbits):
        qrout = QRoutine()
        qbits = qrout.new_wires(nqbits)
        qrout.apply(R_gate.dag(), qbits)
        qrout.apply(P_gate.dag(), qbits[:-1])
        qrout.apply(D_0(nqbits), qbits)
        qrout.apply(P_gate, qbits[:-1])
        qrout.apply(R_gate, qbits)
        return qrout
    UPhi_2 = AbstractGate("UPhi_2", [int])
    UPhi_2.set_circuit_generator(Difusor)
    return UPhi_2(N)

In [ ]:
def DoCircuit(nqbits = 4, Reflexion = True):
    from qat.lang.AQASM import Program
    nbins = 2**nqbits
    a = 0
    b = 1
    #Discretization for the function domain
    centers, probs = get_histogram(p, a, b, nbins)
    #Create Probability loading gate
    P_gate = CreatePG(probs)     
    #Discretizated function to load 
    DiscretizedFunction = f(centers) 
    #Create Function loading gate
    R_gate = CreateLoadFunctionGate(DiscretizedFunction)   
    #Difusor
    D_gate = LoadDifusorGate(nqbits+1, probs, DiscretizedFunction)

    #Quantum Program
    qprog = Program()
    qbits = qprog.qalloc(nqbits+1)
    qprog.apply(P_gate, qbits[:-1])
    qprog.apply(R_gate, qbits)
    if Reflexion:
        qprog.apply(D_gate, qbits)
    
    #Create the circuit from the program
    circuit = qprog.to_circ()
    job = circuit.to_job()
    #Import and create the linear algebra simulator
    linalgqpu = LinAlg()
    #Submit the job to the simulator LinAlg and get the results
    result = linalgqpu.submit(job)
    QP = []
    States = []
    QA = []
    #Print the results
    for sample in result:
        #print("State %s probability %s amplitude %s" % (sample.state, sample.probability, sample.amplitude))
        QP.append(sample.probability)
        States.append(str(sample.state))
        QA.append(sample.amplitude)
    QP = pd.Series(QP, name='Probability')  
    States = pd.Series(States, name='States')  
    QA = pd.Series(QA, name='Amplitude') 
    pdf = pd.concat([States, QP, QA], axis=1)
    return pdf, circuit

In [ ]:
pdfNotReflexion, circuitNR = DoCircuit(Reflexion=False)
pdfReflexion, circuitR = DoCircuit()

In [ ]:
%qatdisplay circuitNR --depth 0

In [ ]:
pdfReflexion

In [ ]:
pdfNotReflexion

In [ ]:
%qatdisplay circuitR --depth 0

Cabe destacar que:
$$\hat{U}_{|\Psi_{2}\rangle } |\Psi_{2}\rangle = (\hat{I} - 2|\Psi_{2}\rangle \langle\Psi_{2}|)|\Psi_{2}\rangle=-|\Psi_{2}\rangle$$

Esto lo podemos utilizar para verificar que hemos implementado bien la puerta

In [ ]:
print('Si aplicamos el Difusor sobre \Psi_2 obtenemos -\Psi_2: {}'.format(
    np.isclose(pdfReflexion['Amplitude'], -pdfNotReflexion['Amplitude']).all()
))